In [ ]:
# 분야 2 | 유선 네트워크 장비의 신속한 조치를 위한 경보 유형 분류

# 아래의 코드는 다음과 같은 순서로 구성되어 있습니다.
# 1. 데이터 로드 및 전처리
# 2. 모델 학습 및 예측
# 3. 결과 제출
# 본 코드는 결과물 제출까지의 이해를 돕기위한 코드로, 반드시 이 구조를 따를 필요는 없습니다.
# 데이터 전처리, 모델의 구성 등에서 다양한 시도를 하시되, 올바른 채점을 위한 최종 결과물의 형태에 유의하시기 바랍니다.

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#데이터 전처리

In [5]:
# 1. 데이터 로드
import pandas as pd

def ppr_data(df):
    # TODO: 데이터 전처리 코드 구현 ---------- #
    df = df.groupby('ticketno').apply(custom_info).reset_index()  # ticketno 기준 병합 및 전처리 수행

    x_df = df.iloc[:, :]
    return x_df

def custom_info(group):  # 임의로 작성된 전처리 코드입니다.
    d = {}
    group.sort_values(by='alarmtime', ascending=True, inplace=True)  # 경보 순서 정렬
    d['alarmmsg_original'] = ' '.join(group['alarmmsg_original'])  # 메시지 단순 병합
    if 'root_cause_type' in group.columns:  # 레이블 추출
        d['root_cause_type'] = group['root_cause_type'].iloc[0]  # 동일한 ticketno는 동일한 root_cause_type을 가짐
    else:
        d['root_cause_type'] = None  # 테스트 세트의 경우 정답 컬럼 없음
    return pd.Series(d, index=['alarmmsg_original', 'root_cause_type'])

train_df = pd.read_csv("/content/drive/MyDrive/제 2회 통신만 안정성 확보를 위한 인공지능 해커톤/Q2_data/Q2_train.csv")
test_df = pd.read_csv("/content/drive/MyDrive/제 2회 통신만 안정성 확보를 위한 인공지능 해커톤/Q2_data/Q2_test.csv")
train_df_filled=train_df.fillna(0)
test_df_filled=test_df.fillna(0)

x_train_df_filled = ppr_data(train_df_filled)
x_test_df_filled= ppr_data(test_df_filled)  # 테스트 세트의 경우 정답 컬럼 없음
x_train_df_filled

,ticketno,alarmmsg_original,root_cause_type
0,14753084.0,PSU-FAIL ETH-LINK-FAIL PSU-FAIL ETH-LINK-FAIL ...,PowerFail
1,14771766.0,ETH-LINK-FAIL PSU-FAIL PSU-FAIL ETH-LINK-FAIL,PowerFail
2,14777089.0,PSU-FAIL PSU-FAIL ETH-LINK-FAIL ETH-LINK-FAIL ...,PowerFail
3,14790052.0,PSU-FAIL PSU-FAIL,PowerFail
4,14879922.0,ETH-LINK-FAIL ETH-LINK-FAIL PSU-FAIL PSU-FAIL ...,PowerFail
...,...,...,...
1109,21792877.0,OPT-LOS,LinkCut
1110,21793984.0,OPT-LOS OPT-LOS,LinkCut
1111,21799077.0,OPT-LOS OPT-LOS DCC-FAIL DCC-FAIL PW-LOC PW-LOC,LinkCut
1112,21809789.0,OPT-LOS OPT-LOS,LinkCut


In [6]:
x_train_df_filled['category_id'] = x_train_df_filled['root_cause_type'].factorize()[0]
from io import StringIO
category_id_df = x_train_df_filled[['root_cause_type', 'category_id']].drop_duplicates().sort_values('category_id')
category_to_id = dict(category_id_df.values)
id_to_category = dict(category_id_df[['root_cause_type', 'category_id']].values)

x_test_df_filled['category_id'] = x_test_df_filled['root_cause_type'].factorize()[0]
from io import StringIO
category_id_df = x_test_df_filled[['root_cause_type', 'category_id']].drop_duplicates().sort_values('category_id')
category_to_id = dict(category_id_df.values)
id_to_category = dict(category_id_df[['root_cause_type', 'category_id']].values)

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(sublinear_tf=True, min_df=5, norm='l2', encoding='latin-1', ngram_range=(1, 8))

features = tfidf.fit_transform(x_test_df_filled.alarmmsg_original).toarray()
labels = x_test_df_filled.category_id
features.shape

(4327, 1885)

In [8]:
x_test_df_filled
x_test_drop=x_test_df_filled.drop(columns=['root_cause_type'])
x_test_drop

,ticketno,alarmmsg_original,category_id
0,15238899.0,PSU-FAIL ETH-LINK-FAIL PSU-FAIL ETH-LINK-FAIL,-1
1,15712444.0,PSU-FAIL PSU-FAIL,-1
2,15723187.0,ETH-NO-RX-TRAFFIC PSU-FAIL PSU-FAIL,-1
3,15737103.0,PSU-FAIL ETH-LINK-FAIL ETH-LINK-FAIL PSU-FAIL,-1
4,15737132.0,ETH-LINK-FAIL PSU-FAIL ETH-LINK-FAIL PSU-FAIL,-1
...,...,...,...
4322,22015278.0,DDM_RX_PWR_HIGH,-1
4323,22015300.0,Loss Of Signal,-1
4324,23818326.0,Server Signal Failure Loss Of Signal Server Si...,-1
4325,23819373.0,Server Signal Failure,-1


In [9]:
result_train = x_train_df_filled["alarmmsg_original"].str.split()
data=result_train
flat_data = [item for sublist in data for item in sublist]
set_flat_data=set(flat_data)
print(set_flat_data)
print(len(set_flat_data))

result_test = x_test_drop["alarmmsg_original"].str.split()
data_test=result_test
flat_data_test = [item for sublist in data_test for item in sublist]
set_flat_data_test=set(flat_data_test)
print(set_flat_data_test)
print(len(set_flat_data_test))
# Use a set to keep track of unique values

# Find items in test set that are not in train set
items_in_test_not_in_train = set_flat_data_test - set_flat_data

# Find items in train set that are not in test set
items_in_train_not_in_test = set_flat_data - set_flat_data_test
items_in_test_not_in_train
items_in_train_not_in_test

{'PSU-REM', 'BATT-LOW', 'ETH-NO-TX-TRAFFIC', 'BOOTING', 'DCC-FAIL', 'TRK-CONN-MIS', 'OS_MISMATCH', 'ETH-LINK-FAIL', 'Restarted', 'AUTONEGO_MISMATCH', 'ProtectionSwitched', 'NVRAM-FAIL', 'FAN-FAIL', 'OPT-LOS', 'PortShutdown', 'LSP-LOC', 'OPT-PWR-LOW', 'TDM-PW-RMT_FAIL', 'OPT-MIS', 'ETH-NO-RX-TRAFFIC', 'ETH-ERR', 'OPT-REMOVE', 'UNIT-IPC-FAIL', 'PSU-FAIL', 'PW-LOC', 'FAN-48V-FAIL', 'LINK_FAIL_MANUAL', 'QL_FAIL', '48V-FAIL', 'UNIT-REM', 'TDM-PW-LOF'}
31
{'(BY', 'Raise', 'DC_INPUT_A_FAIL', 'UNIT_OUT', 'Condition:', 'Client', 'TRK-CONN-MIS', 'Mng', '(LLCF)', 'UNIT_FAIL', 'Continuity', 'RESET)', 'Procedure', 'STM1_LOS', 'OPT-PWR-LOW', 'Primary', 'Degrade', 'Signal', 'OPT-PWR-HIGH', 'Connectivity', 'Synchronization', 'ETH-ERR', 'Unavailable', 'PW-LOC', 'ETHER_TCA_CRC_1D', 'RDI', 'Abnormal', 'QL_FAIL', 'NE', 'Laser', 'Not', '48V-FAIL', 'Node', 'Improper', 'DDM_RX_PWR_HIGH', 'Via', 'Isolation', 'OAM', 'Keeping', 'MEP_PWE_LOC', 'Power', 'OAMLoss', '(LOCAL_FAULT)', 'Rx', 'BATTERY_FAIL', 'OPT-LOS',

{'BATT-LOW',
 'FAN-48V-FAIL',
 'FAN-FAIL',
 'NVRAM-FAIL',
 'OS_MISMATCH',
 'PSU-REM',
 'PortShutdown',
 'ProtectionSwitched',
 'Restarted',
 'TDM-PW-LOF',
 'UNIT-IPC-FAIL',
 'UNIT-REM'}

In [10]:
data=x_train_df_filled
count_opt_remove_unitfail = len(data[(data['alarmmsg_original'].str.contains('BATT-LOW')) & (data['root_cause_type'] == 'UnitFail')])

print(count_opt_remove_unitfail)

0


In [11]:
from sklearn.feature_selection import chi2
import numpy as np

N = 5
for root_cause_type, category_id in sorted(category_to_id.items()):
  features_chi2 = chi2(features, labels == category_id)
  indices = np.argsort(features_chi2[0])
  feature_names = np.array(tfidf.get_feature_names_out())[indices]
  unigrams = [v for v in feature_names if len(v.split(' ')) == 3]
  bigrams = [v for v in feature_names if len(v.split(' ')) == 2]
  fourgrams = [v for v in feature_names if len(v.split(' ')) == 4]

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(sublinear_tf=True, min_df=5, norm='l2', encoding='latin-1', ngram_range=(1, 8))

features = tfidf.fit_transform(x_train_df_filled.alarmmsg_original).toarray() #train
labels = x_train_df_filled.category_id #label
features.shape

(1114, 522)

In [13]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB

X_train, X_test, y_train, y_test = train_test_split(x_train_df_filled['alarmmsg_original'], x_train_df_filled['root_cause_type'], random_state = 0)
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

clf = MultinomialNB().fit(X_train_tfidf, y_train)

In [14]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC

from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
import xgboost as xgb

model = xgb.XGBClassifier()

CV = 5
entries = []

model_name = model.__class__.__name__
accuracies = cross_val_score(model, features, labels, scoring='accuracy', cv=CV)
for fold_idx, accuracy in enumerate(accuracies):
  entries.append((model_name, fold_idx, accuracy))
cv_df = pd.DataFrame(entries, columns=['model_name', 'fold_idx', 'accuracy'])

In [15]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((835,), (835,), (279,), (279,))

In [16]:
labels.shape, features.shape

((1114,), (1114, 522))

In [17]:
text_features = tfidf.transform(x_test_drop['alarmmsg_original']) #pred f

In [18]:
import xgboost as xgb

dtrain = xgb.DMatrix(data=features, label=labels)
dtest = xgb.DMatrix(data=text_features)

In [19]:
params = {
    'max_depth': 3,
    'objective': 'multi:softmax',  # error evaluation for multiclass training
    'num_class': 10,
    'n_gpus': 0
}

In [20]:
dtest = xgb.DMatrix(data=text_features)
bst = xgb.train(params, dtrain)
pred = bst.predict(dtest)
pred.sum()
y_test.sum()
len(y_test)

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [01:47:35] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "n_gpus" } are not used.

  warnings.warn(smsg, UserWarning)


279

In [21]:
len(pred)

4327

In [22]:
id_to_root_cause = {
    0: "PowerFail",
    1: "UnitFail",
    2: "LinkCut",
    # Add more mappings if needed
}
predicted_root_cause = [id_to_root_cause[cat_id] for cat_id in pred]

# Print the predictions with root_cause_type
print(predicted_root_cause)

['UnitFail', 'UnitFail', 'UnitFail', 'UnitFail', 'UnitFail', 'UnitFail', 'UnitFail', 'PowerFail', 'UnitFail', 'UnitFail', 'UnitFail', 'UnitFail', 'UnitFail', 'UnitFail', 'UnitFail', 'UnitFail', 'UnitFail', 'UnitFail', 'UnitFail', 'UnitFail', 'UnitFail', 'UnitFail', 'UnitFail', 'UnitFail', 'UnitFail', 'UnitFail', 'UnitFail', 'UnitFail', 'UnitFail', 'UnitFail', 'UnitFail', 'UnitFail', 'UnitFail', 'UnitFail', 'UnitFail', 'UnitFail', 'UnitFail', 'UnitFail', 'UnitFail', 'UnitFail', 'UnitFail', 'UnitFail', 'UnitFail', 'UnitFail', 'UnitFail', 'UnitFail', 'UnitFail', 'UnitFail', 'UnitFail', 'UnitFail', 'UnitFail', 'UnitFail', 'UnitFail', 'UnitFail', 'UnitFail', 'UnitFail', 'UnitFail', 'UnitFail', 'UnitFail', 'UnitFail', 'UnitFail', 'UnitFail', 'UnitFail', 'UnitFail', 'UnitFail', 'UnitFail', 'UnitFail', 'UnitFail', 'UnitFail', 'UnitFail', 'UnitFail', 'UnitFail', 'UnitFail', 'UnitFail', 'UnitFail', 'UnitFail', 'UnitFail', 'UnitFail', 'UnitFail', 'UnitFail', 'UnitFail', 'UnitFail', 'UnitFail', 'U

In [23]:
x_test_drop['root_cause_type']=predicted_root_cause
x_test_drop

,ticketno,alarmmsg_original,category_id,root_cause_type
0,15238899.0,PSU-FAIL ETH-LINK-FAIL PSU-FAIL ETH-LINK-FAIL,-1,UnitFail
1,15712444.0,PSU-FAIL PSU-FAIL,-1,UnitFail
2,15723187.0,ETH-NO-RX-TRAFFIC PSU-FAIL PSU-FAIL,-1,UnitFail
3,15737103.0,PSU-FAIL ETH-LINK-FAIL ETH-LINK-FAIL PSU-FAIL,-1,UnitFail
4,15737132.0,ETH-LINK-FAIL PSU-FAIL ETH-LINK-FAIL PSU-FAIL,-1,UnitFail
...,...,...,...,...
4322,22015278.0,DDM_RX_PWR_HIGH,-1,UnitFail
4323,22015300.0,Loss Of Signal,-1,UnitFail
4324,23818326.0,Server Signal Failure Loss Of Signal Server Si...,-1,UnitFail
4325,23819373.0,Server Signal Failure,-1,UnitFail


#예측 후처리

In [24]:
x_test_drop.loc[x_test_drop['alarmmsg_original'].str.contains('BATTERY_FAIL'), 'root_cause_type'] = 'PowerFail'
x_test_drop.loc[x_test_drop['alarmmsg_original'].str.contains('BATT_ENV_FAIL'), 'root_cause_type'] = 'PowerFail'
x_test_drop.loc[x_test_drop['alarmmsg_original'].str.contains('UNIT_FAIL'), 'root_cause_type'] = 'UnitFail'
x_test_drop.loc[x_test_drop['alarmmsg_original'].str.contains('Loss Of Signal'), 'root_cause_type'] = 'LinkCut'

In [25]:
#x_test_drop.drop('alarmmsg_original', axis=1, inplace=True)
x_test_drop.drop('category_id', axis=1, inplace=True)
x_test_drop

,ticketno,alarmmsg_original,root_cause_type
0,15238899.0,PSU-FAIL ETH-LINK-FAIL PSU-FAIL ETH-LINK-FAIL,UnitFail
1,15712444.0,PSU-FAIL PSU-FAIL,UnitFail
2,15723187.0,ETH-NO-RX-TRAFFIC PSU-FAIL PSU-FAIL,UnitFail
3,15737103.0,PSU-FAIL ETH-LINK-FAIL ETH-LINK-FAIL PSU-FAIL,UnitFail
4,15737132.0,ETH-LINK-FAIL PSU-FAIL ETH-LINK-FAIL PSU-FAIL,UnitFail
...,...,...,...
4322,22015278.0,DDM_RX_PWR_HIGH,UnitFail
4323,22015300.0,Loss Of Signal,LinkCut
4324,23818326.0,Server Signal Failure Loss Of Signal Server Si...,LinkCut
4325,23819373.0,Server Signal Failure,UnitFail


#결과 제출


In [29]:
# 3. 결과 제출
# 본 코드는 제출되는 파일의 형태에 대한 가이드로, 반드시 아래 구조를 따를 필요 없이 자유롭게 코드를 작성해도 무방합니다.
# 제출 포맷에 대해선 data/Q2_label_sample.csv를 참조하세요.
#
# 분야 2의 경우, 전표(ticket) 하나에 하나의 근원장애(root_cause_type)을 매칭해야 합니다.
#   주의: 경보(alarm) 개수와 전표(ticket) 개수는 다르며, 예측할 대상은 전표입니다.
#   주의: ticketno 컬럼 기준으로 오름차순 정렬이 필요합니다.
# 분야 2의 제출 파일은 2개 컬럼 [ticketno, root_cause_type]을 가져야 합니다.

def submitResult(pred):
    try:
        label = pd.read_csv('/content/drive/MyDrive/제 2회 통신만 안정성 확보를 위한 인공지능 해커톤/Q2_data/Q2_label_sample.csv')
        # ticketno 순서와 개수가 일치하는지 확인
        if (label['ticketno'] == pred['ticketno']).all():
            print("Check: ticketno 순서와 샘플 수가 일치합니다.")
        else:
            print("Warning: 테스트 세트와 모델 예측의 ticketno가 일치하지 않습니다.")
            return

        pred.to_csv('Q2_submitResult.csv', index=False)
        print("Done : Q2_submitResult.csv 파일로 저장되었습니다.")
    except Exception as e:
        # 예외가 발생한 경우 오류 메시지 출력
        print("Error:", e)

submitResult(x_test_drop)

Check: ticketno 순서와 샘플 수가 일치합니다.
Done : Q2_submitResult.csv 파일로 저장되었습니다.
